# Importing packages

In [2]:
import json
import codecs
import math
import numpy as np
import utils
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from pylab import *

# Defining required function and objects

Objects to load training and test data

In [3]:
class Data:
    def __init__(self, domain, data_dir, flag_word2vec, label_dict, seq_max_len, flag_addition_corpus,
     flag_change_file_structure, negative_weight, positive_weight, neutral_weight, flag_use_sentiment_embedding):
        self.seq_max_len = seq_max_len
        self.label_dict = label_dict
        self.domain = domain
        self.data_dir = data_dir
        self.flag_word2vec = flag_word2vec
        self.flag_addition_corpus = flag_addition_corpus
        self.flag_change_file_structure = flag_change_file_structure
        self.negative_weight = negative_weight
        self.positive_weight = positive_weight
        self.neutral_weight = neutral_weight
        self.flag_use_sentiment_embedding = flag_use_sentiment_embedding

        self.train_data, self.train_mask, self.train_binary_mask, self.train_label, self.train_seq_len, self.train_sentiment_for_word, \
        self.test_data, self.test_mask, self.test_binary_mask, self.test_label, self.test_seq_len, self.test_sentiment_for_word, \
        self.word_dict, self.word_dict_rev, self.embedding, aspect_list = utils.load_data(
            self.domain,
            self.data_dir,
            self.flag_word2vec,
            self.label_dict,
            self.seq_max_len,
            self.flag_addition_corpus,
            self.flag_change_file_structure,
            self.negative_weight,
            self.positive_weight,
            self.neutral_weight,
            self.flag_use_sentiment_embedding
        )

        self.nb_sample_train = len(self.train_data)
        
        self.x_test = list()
        for i in range(len(self.test_data)):
            sentence = list()
            for word_id in self.test_data[i]:
                sentence.append(self.embedding[word_id])
            self.x_test.append(sentence)
            
        self.x_train = list()
        for i in range(len(self.train_data)):
            sentence = list()
            for word_id in self.train_data[i]:
                sentence.append(self.embedding[word_id])
            self.x_train.append(sentence)

    def parse_data(self, sentences):
        self.test_data = list()
        self.test_binary_mask = list()

        for sentence in sentences:
            words = sentence.split(' ')
            data_tmp = list()
            binary_mask_tmp = list()
            count_len = 0
            for word in words:
                if (word in word_dict.keys() and count_len < seq_max_len):
                    data_tmp.append(self.word_dict[word])
                    binary_mask_tmp.append(1.0)
                    count_len = count_len + 1
    
            for _ in range(self.seq_max_len - count_len):
                data_tmp.append(word_dict['<padding>'])
                binary_mask_tmp.append(0.)

            self.test_data.append(data_tmp)
            self.test_binary_mask.append(binary_mask_tmp)

        self.x_test = list()
        for i in range(len(self.test_data)):
            sentence = list()
            for word_id in self.test_data[i]:
                sentence.append(self.embedding[word_id])
            self.x_test.append(sentence)

Object to create model and defining funtions for training and evaluating

In [4]:
class Model:
    def __init__(self, batch_size, seq_max_len, nb_sentiment_label, nb_sentiment_for_word, embedding_size, nb_linear_inside,
        nb_lstm_inside, layers, TRAINING_ITERATIONS, LEARNING_RATE, WEIGHT_DECAY, flag_train, flag_use_sentiment_for_word, sess):
        self.batch_size = batch_size
        self.seq_max_len = seq_max_len
        self.nb_sentiment_label = nb_sentiment_label
        self.nb_sentiment_for_word = nb_sentiment_for_word
        self.embedding_size = embedding_size
        self.nb_linear_inside = nb_linear_inside
        self.nb_lstm_inside = nb_lstm_inside
        self.layers = layers
        self.TRAINING_ITERATIONS = TRAINING_ITERATIONS
        self.LEARNING_RATE = LEARNING_RATE
        self.WEIGHT_DECAY = WEIGHT_DECAY
        self.flag_train = flag_train
        self.flag_use_sentiment_for_word = flag_use_sentiment_for_word
        self.sess = sess

    def modeling(self):
        self.tf_X_train = tf.placeholder(tf.float32, \
            shape = [None, self.seq_max_len, self.embedding_size - int(self.flag_use_sentiment_for_word) * self.nb_sentiment_for_word])
        self.tf_X_sent_for_word = tf.placeholder(tf.int64, shape = [None, self.seq_max_len])
        self.tf_X_train_mask = tf.placeholder(tf.float32, shape = [None, self.seq_max_len])
        self.tf_X_binary_mask = tf.placeholder(tf.float32, shape = [None, self.seq_max_len])
        self.tf_y_train = tf.placeholder(tf.int64, shape = [None, self.seq_max_len])
        self.tf_X_seq_len = tf.placeholder(tf.int64, shape = [None])
        self.keep_prob = tf.placeholder(tf.float32)
        

        self.ln_w = tf.Variable(tf.truncated_normal([self.embedding_size, self.nb_linear_inside], stddev = math.sqrt(3.0 / (self.embedding_size + self.nb_linear_inside))))
        self.ln_b = tf.Variable(tf.zeros([self.nb_linear_inside]))
         
        self.sent_w = tf.Variable(tf.truncated_normal([self.nb_lstm_inside, self.nb_sentiment_label],
                                                 stddev = math.sqrt(3.0 / self.nb_lstm_inside + self.nb_sentiment_label)))
        self.sent_b = tf.Variable(tf.zeros([self.nb_sentiment_label]))

        y_labels = tf.one_hot(self.tf_y_train,
                              self.nb_sentiment_label,
                              on_value = 1.0,
                              off_value = 0.0,
                              axis = -1)
         

        if (self.flag_use_sentiment_for_word):
            X_sent_for_word = tf.one_hot(self.tf_X_sent_for_word, self.nb_sentiment_for_word,
                                 on_value = 20.0,
                                 off_value = 10.0,
                                 axis = -1)

            X_train = tf.concat([self.tf_X_train, X_sent_for_word], 2)
            X_train = tf.transpose(X_train, [1, 0, 2])
        else:
            X_train = tf.transpose(self.tf_X_train, [1, 0, 2])

        # Reshaping to (n_steps * batch_size, n_input)
        X_train = tf.reshape(X_train, [-1, self.embedding_size])
        X_train = tf.nn.relu(tf.add(tf.matmul(X_train, self.ln_w), self.ln_b))
        X_train = tf.nn.dropout(X_train, self.keep_prob)
        X_train = tf.split(axis = 0, num_or_size_splits = self.seq_max_len, value = X_train)
        
        # bidirection lstm
        # Creating the forward and backwards cells
        lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.nb_lstm_inside, forget_bias = 0.8)
        lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.nb_lstm_inside, forget_bias = 0.8)
        # Get lstm cell output
        outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell,
                                                     lstm_bw_cell,
                                                     X_train,
                                                     dtype='float32')

        output_fw, output_bw = tf.split(outputs, [self.nb_lstm_inside, self.nb_lstm_inside], 2)
        sentiment = tf.reshape(tf.add(output_fw, output_bw), [-1, self.nb_lstm_inside])
        
        # sentiment = tf.reshape(outputs, [-1, 2 * self.nb_lstm_inside]) 
        sentiment = tf.nn.dropout(sentiment, self.keep_prob)
        sentiment = tf.add(tf.matmul(sentiment, self.sent_w), self.sent_b)
        sentiment = tf.split(axis = 0, num_or_size_splits = self.seq_max_len, value = sentiment)

        # Change back dimension to [batch_size, n_step, n_input]
        sentiment = tf.stack(sentiment)
        sentiment = tf.transpose(sentiment, [1, 0, 2])
        sentiment = tf.multiply(sentiment, tf.expand_dims(self.tf_X_binary_mask, 2))

        self.cross_entropy = tf.reduce_mean(tf.multiply(tf.nn.softmax_cross_entropy_with_logits(logits = sentiment, labels = y_labels), self.tf_X_train_mask))
        
        regularization = self.WEIGHT_DECAY * sum(
            tf.nn.l2_loss(tf_var)
                for tf_var in tf.trainable_variables()
                if not ("noreg" in tf_var.name or "Bias" in tf_var.name)
        )
        self.cross_entropy = self.cross_entropy + regularization
        
        self.prediction = tf.argmax(tf.nn.softmax(sentiment), 2)
        self.correct_prediction = tf.reduce_sum(tf.multiply(tf.cast(tf.equal(self.prediction, self.tf_y_train), tf.float32), self.tf_X_binary_mask))
        self.global_step = tf.Variable(0, trainable = True)
        self.learning_rate = tf.train.exponential_decay(self.LEARNING_RATE, self.global_step, 1000, 0.65, staircase = True)
        # self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.cross_entropy, global_step = self.global_step)
        self.optimizer = tf.train.AdagradOptimizer(self.learning_rate).minimize(self.cross_entropy)

        self.saver = tf.train.Saver()
        self.init = tf.global_variables_initializer()


    def load_model(self):
        self.saver.restore(self.sess, '../ckpt/se-apect-v0.ckpt')

    def save_model(self):
        self.saver.save(self.sess, '../ckpt/se-apect-v0.ckpt')

    def predict(self, data):
        prediction_test = self.sess.run(self.prediction, 
                          feed_dict={self.tf_X_train: np.asarray(data.x_test),
                                     self.tf_X_binary_mask: np.asarray(data.test_binary_mask),
                                     self.tf_X_seq_len: np.asarray(data.test_seq_len),
                                     self.tf_X_sent_for_word: np.asarray(data.test_sentiment_for_word),
                                     self.keep_prob: 1.0})


        ret = list()
        for i in range(len(data.test_data)):
                data_sample = ''
                for j in range(len(data.test_data[i])):
                    if data.word_dict_rev[data.test_data[i][j]] == '<unk>':
                        continue
                    elif data.test_binary_mask[i][j] > 0.:
                        data_sample = data_sample + data.word_dict_rev[data.test_data[i][j]] + \
                         '(predict ' + str(prediction_test[i][j]) + ') '
                    else:
                        data_sample = data_sample + data.word_dict_rev[data.test_data[i][j]] + ' '
                ret.append(data_sample.replace('<padding>', '').strip())
        return ret


    def evaluate(self, data, flag_write_to_file, flag_train):
        if (not flag_train):
            self.load_model()

        correct_prediction_test, prediction_test = self.sess.run([self.correct_prediction, self.prediction], 
                                              feed_dict={self.tf_X_train: np.asarray(data.x_test),
                                                         self.tf_X_binary_mask: np.asarray(data.test_binary_mask),
                                                         self.tf_X_seq_len: np.asarray(data.test_seq_len),
                                                         self.tf_X_sent_for_word: np.asarray(data.test_sentiment_for_word),
                                                         self.tf_y_train: np.asarray(data.test_label),
                                                         self.keep_prob: 1.0})

        print('test accuracy => %.3f' %(float(correct_prediction_test)/np.sum(data.test_binary_mask)))

        if float(correct_prediction_test)/np.sum(data.test_binary_mask) > 0.809:
            self.save_model()

        if (flag_write_to_file):
            f_result = codecs.open('../result/result.txt', 'w', 'utf-8')
            f_result.write('#---------------------------------------------------------------------------------------------------------#\n')
            f_result.write('#\t author: BinhDT\n')
            f_result.write('#\t test accuracy %.2f\n' %(float(correct_prediction_test)*100/np.sum(np.asarray(data.test_binary_mask) > 0.)))
            f_result.write('#\t 1:positive, 0:neutral, 2:negative\n')
            f_result.write('#---------------------------------------------------------------------------------------------------------#\n')

            for i in range(len(data.test_data)):
                data_sample = ''
                for j in range(len(data.test_data[i])):
                    if data.word_dict_rev[data.test_data[i][j]] == '<unk>':
                        continue
                    elif data.test_binary_mask[i][j] > 0.:
                        data_sample = data_sample + data.word_dict_rev[data.test_data[i][j]] + '(label ' + str(data.test_label[i][j]) + \
                         '|predict ' + str(prediction_test[i][j]) + ') '
                    else:
                        data_sample = data_sample + data.word_dict_rev[data.test_data[i][j]] + ' '
                f_result.write('%s\n' %data_sample.replace('<padding>', '').strip())

            f_result.close()

    def train(self, data):
        self.sess.run(self.init)
        # self.load_model()
        loss_list = list()
        accuracy_list = list()

        for it in range(self.TRAINING_ITERATIONS):
            # generate batch (x_train, y_train, seq_lengths_train)
            if (it * self.batch_size % data.nb_sample_train + self.batch_size < data.nb_sample_train):
                index = it * self.batch_size % data.nb_sample_train
            else:
                index = data.nb_sample_train - self.batch_size
            

            self.sess.run(self.optimizer, 
                          feed_dict={self.tf_X_train: np.asarray(data.x_train[index : index + self.batch_size]),
                                     self.tf_X_train_mask: np.asarray(data.train_mask[index : index + self.batch_size]),
                                     self.tf_X_binary_mask: np.asarray(data.train_binary_mask[index : index + self.batch_size]),
                                     self.tf_X_seq_len: np.asarray(data.train_seq_len[index : index + self.batch_size]),
                                     self.tf_X_sent_for_word: np.asarray(data.train_sentiment_for_word[index : index + self.batch_size]),
                                     self.tf_y_train: np.asarray(data.train_label[index : index + self.batch_size]),
                                     self.keep_prob: 0.5})

            if it % (len(data.x_train) // self.batch_size) == 0:
                print(it)
                self.evaluate(data, it + 100 >= self.TRAINING_ITERATIONS, self.flag_train)
                '''
                correct_prediction_train, cost_train = self.sess.run([self.correct_prediction, self.cross_entropy], 
                                                  feed_dict={self.tf_X_train: np.asarray(data.x_train),
                                                             self.tf_X_train_mask: np.asarray(data.train_mask),
                                                             self.tf_X_binary_mask: np.asarray(data.train_binary_mask),
                                                             self.tf_X_seq_len: np.asarray(data.train_seq_len),
                                                             self.tf_X_sent_for_word: np.asarray(data.train_sentiment_for_word),
                                                             self.tf_y_train: np.asarray(data.train_label),
                                                             self.keep_prob: 0.8})
                
                print('training_accuracy => %.3f, cost value => %.5f for step %d' % \
                (float(correct_prediction_train)/np.sum(np.asarray(data.train_binary_mask)), cost_train, it))
                
                loss_list.append(cost_train)
                accuracy_list.append(float(correct_prediction_train)/np.sum(np.asarray(data.train_binary_mask)))
                
                _, ax1 = plt.subplots()
                ax2 = ax1.twinx()
                ax1.plot(loss_list)
                ax2.plot(accuracy_list, 'r')
                ax1.set_xlabel('epoch')
                ax1.set_ylabel('train loss')
                ax2.set_ylabel('train accuracy')
                ax1.set_title('train accuracy and loss')
                ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
                plt.savefig('accuracy_loss.eps', format='eps', dpi=150)
                plt.close()
                '''


#         self.sess.close()

# Main

Basic

In [5]:
batch_size = 128
nb_sentiment_label = 3
nb_sentiment_for_word = 3
embedding_size = 100
nb_lstm_inside = 256
layers = 1
TRAINING_ITERATIONS = 5000
LEARNING_RATE = 0.1
WEIGHT_DECAY = 0.0001
label_dict = {
    'aspositive' : 1,
    'asneutral' : 0,
    'asnegative': 2
}
data_dir = '../data/ABSA_SemEval2014/'
domain = 'Restaurants'

Required for resturant 2014

In [6]:
seq_max_len = 35
nb_linear_inside = 512

Others

In [7]:
flag_word2vec = False
flag_addition_corpus = False
flag_change_file_structure = False
flag_use_sentiment_embedding = False
flag_use_sentiment_for_word = True
flag_train = True

In [8]:
negative_weight = 1.0
positive_weight = 1.0
neutral_weight = 1.0

#### Loading data

In [9]:
data = Data(domain, data_dir, flag_word2vec, label_dict, seq_max_len, flag_addition_corpus, flag_change_file_structure, negative_weight, positive_weight, neutral_weight, flag_use_sentiment_embedding)

pos: 1431
neu: 474
neg: 539
len of train data is 2021
len of test data is 606


#### Training

In [10]:
sess = tf.Session()
if (flag_use_sentiment_for_word):
    embedding_size = embedding_size + nb_sentiment_for_word

model = Model(batch_size, seq_max_len, nb_sentiment_label, nb_sentiment_for_word, embedding_size, nb_linear_inside, nb_lstm_inside, layers, TRAINING_ITERATIONS, LEARNING_RATE, WEIGHT_DECAY, flag_train, flag_use_sentiment_for_word,sess)

model.modeling()
model.train(data)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
C

c:\users\aritra banerjee\envs\dl\lib\site-packages\ipykernel_launcher.py:88: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
0
test accuracy => 0.657
15
test accuracy => 0.149
30
test accuracy => 0.657
45
test accuracy => 0.304
60
test accuracy => 0.657
75
test accuracy => 0.657
90
test accuracy => 0.657
105
test accuracy => 0.623
120
test accuracy => 0.657
135
test accuracy => 0.657
150
test accuracy => 0.657
165
test accuracy => 0.657
180
test accuracy => 0.657
195
test accuracy => 0.657
210
test accuracy => 0.657
225
test accuracy => 0.657
240
test accuracy => 0.657
255
test accuracy => 0.657
270
test accuracy => 0.657
285
test accuracy => 0.657
300
test accuracy => 0.657
315
test accuracy => 0.657
330
test accuracy => 0.657
345
test accuracy => 0.657
360
test accuracy => 0.657
375
test accuracy => 0.657
390
test accuracy => 0.657
405
test accuracy => 0.657
420
test accuracy => 0.657
435
test accuracy => 0.657
450
test accuracy => 0.657
465
test accuracy => 0.657
480
test accuracy => 0.657

4245
test accuracy => 0.728
4260
test accuracy => 0.728
4275
test accuracy => 0.729
4290
test accuracy => 0.722
4305
test accuracy => 0.694
4320
test accuracy => 0.720
4335
test accuracy => 0.720
4350
test accuracy => 0.718
4365
test accuracy => 0.716
4380
test accuracy => 0.726
4395
test accuracy => 0.729
4410
test accuracy => 0.729
4425
test accuracy => 0.722
4440
test accuracy => 0.726
4455
test accuracy => 0.728
4470
test accuracy => 0.728
4485
test accuracy => 0.718
4500
test accuracy => 0.726
4515
test accuracy => 0.724
4530
test accuracy => 0.722
4545
test accuracy => 0.731
4560
test accuracy => 0.731
4575
test accuracy => 0.729
4590
test accuracy => 0.720
4605
test accuracy => 0.713
4620
test accuracy => 0.724
4635
test accuracy => 0.722
4650
test accuracy => 0.729
4665
test accuracy => 0.724
4680
test accuracy => 0.724
4695
test accuracy => 0.728
4710
test accuracy => 0.728
4725
test accuracy => 0.731
4740
test accuracy => 0.729
4755
test accuracy => 0.728
4770
test accuracy =

In [26]:
def predictRAW(model, data):
    prediction_test = model.sess.run(model.prediction, 
                      feed_dict={model.tf_X_train: np.asarray(data.x_test),
                                 model.tf_X_binary_mask: np.asarray(data.test_binary_mask),
                                 model.tf_X_seq_len: np.asarray(data.test_seq_len),
                                 model.tf_X_sent_for_word: np.asarray(data.test_sentiment_for_word),
                                 model.keep_prob: 1.0})


    ret = list()
    for i in range(len(data.test_data)):
            data_sample = ''
            for j in range(len(data.test_data[i])):
                if data.word_dict_rev[data.test_data[i][j]] == '<unk>':
                    continue
                elif data.test_binary_mask[i][j] > 0.:
                    data_sample = data_sample + data.word_dict_rev[data.test_data[i][j]] + \
                     '(predict ' + str(prediction_test[i][j]) + ') '
                else:
                    data_sample = data_sample + data.word_dict_rev[data.test_data[i][j]] + ' '
            ret.append(data_sample.replace('<padding>', '').strip())
    return ret

In [28]:
# predictRAW(model, data)